In [1]:
#run once - small sets of vectors for development (low-D, high minimum frequency threshold so relatively few concepts)
!wget https://www.dropbox.com/s/3f5qg9nfop2l45e/semanticvectors.bin
!wget https://www.dropbox.com/s/r8r8ffw9rnjmmo0/permutationvectors.bin
!wget https://www.dropbox.com/s/nf28cvp1ocvyh1s/elementalvectors.bin

--2019-07-12 15:09:20--  https://www.dropbox.com/s/3f5qg9nfop2l45e/semanticvectors.bin
Resolving www.dropbox.com... 162.125.1.1
Connecting to www.dropbox.com|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/3f5qg9nfop2l45e/semanticvectors.bin [following]
--2019-07-12 15:09:20--  https://www.dropbox.com/s/raw/3f5qg9nfop2l45e/semanticvectors.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc86132fd8b45368eff7a6348225.dl.dropboxusercontent.com/cd/0/inline/Aknl3Db8rJ2q7-2t6zGcM0NlnPFaBprJQfzhCsFve2Uf3Kcjgi7uzUTx-16kSzWq-Bo7_O84TTxFzvv8sqLnZ92V-Un_yQtZTXMgpnvjnximLg/file# [following]
--2019-07-12 15:09:20--  https://uc86132fd8b45368eff7a6348225.dl.dropboxusercontent.com/cd/0/inline/Aknl3Db8rJ2q7-2t6zGcM0NlnPFaBprJQfzhCsFve2Uf3Kcjgi7uzUTx-16kSzWq-Bo7_O84TTxFzvv8sqLnZ92V-Un_yQtZTXMgpnvjnximLg/file
Resolving uc86132fd8b45368eff7a6348225.dl.dropboxusercontent.

In [9]:
!ls
import copy
import sys
sys.path.append("../")
import permutations.semvec_utils as sv
import permutations.constants as c
import permutations.sparse_permutations as sp
import permutations.semvec_utils as sv

elementalvectors.bin    permutationvectors.bin  semvec_utils_demo.ipynb
permutations_demo.ipynb semanticvectors.bin


In [10]:
#run once 
semanticvectors = sv.readfile('semanticvectors.bin')
elementalvectors = sv.readfile('elementalvectors.bin')
permutation_cache = sv.readfile('permutationvectors.bin')


In [11]:
#nearest neighbor by term or by vector
print(sv.get_k_vec_neighbors(semanticvectors,'docetaxel',10))
print(sv.get_k_neighbors(semanticvectors,sv.getvector(semanticvectors,'docetaxel'),10))

['docetaxel', 'gemcitabine', 'etoposide', 'paclitaxel', 'carboplatin', 'erlotinib', 'gefitinib', 'capecitabine', 'cetuximab', 'fluorouracil']
['docetaxel', 'gemcitabine', 'etoposide', 'paclitaxel', 'carboplatin', 'erlotinib', 'gefitinib', 'capecitabine', 'cetuximab', 'fluorouracil']


In [12]:
#single predicate search
semvec=copy.copy(sv.getvector(semanticvectors,'prozac'))
permvec=sp.permute_vector(sv.getvector(permutation_cache,'_ISA'),semvec)
results=sv.get_k_neighbors(elementalvectors, permvec, 10)
print(results)

['dopamine_antagonists', 'antidepressive_agents', 'monoamine_oxidase_inhibitors', 'serotonin_uptake_inhibitors', 'reuptake_inhibitors', 'antipsychotic_agents', 'selective_serotonin_re-uptake_inhibitor', 'antidepressive_agents,_tricyclic', 'atypical_antipsychotic', 'serotonin_agents']


In [13]:
#double predicate search - not sure why plants and cattle show up, but the others make sense
semvec=copy.copy(sv.getvector(semanticvectors,'prozac'))
permvec=sp.permute_vector(sv.getvector(permutation_cache,'_ISA'),semvec)
permvec=sp.permute_vector(sv.getvector(permutation_cache,'TREATS-INV'),permvec)
results=sv.get_k_neighbors(semanticvectors, permvec, 10)
print(results)

['plants', 'obsessive-compulsive_disorder', 'phobia,_social', 'depression,_bipolar', 'schizoaffective_disorder', 'sleeplessness', 'relationships', 'cattle', 'chronic_schizophrenia', 'country']


In [14]:
#run once
!wget https://www.dropbox.com/s/53x384ca2ehhcnd/semanticvectors_1024.bin

--2019-07-12 15:11:24--  https://www.dropbox.com/s/53x384ca2ehhcnd/semanticvectors_1024.bin
Resolving www.dropbox.com... 162.125.1.1
Connecting to www.dropbox.com|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/53x384ca2ehhcnd/semanticvectors_1024.bin [following]
--2019-07-12 15:11:24--  https://www.dropbox.com/s/raw/53x384ca2ehhcnd/semanticvectors_1024.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc054c07c66c00d1e24016e6b068.dl.dropboxusercontent.com/cd/0/inline/AkkPYTEtKg9rsvYc1_yJZpwrN9cvoSPkDpirL9fZI8D405vusSlkSy9zs3Gh0ww69arazDATxUcn5aPwDDgkJagLwY2CkCKfmzFltIA8OYIYFA/file# [following]
--2019-07-12 15:11:24--  https://uc054c07c66c00d1e24016e6b068.dl.dropboxusercontent.com/cd/0/inline/AkkPYTEtKg9rsvYc1_yJZpwrN9cvoSPkDpirL9fZI8D405vusSlkSy9zs3Gh0ww69arazDATxUcn5aPwDDgkJagLwY2CkCKfmzFltIA8OYIYFA/file
Resolving uc054c07c66c00d1e24016e6b068.dl.drop

In [15]:
vecs=sv.readfile('semanticvectors_1024.bin')


In [16]:
print(sv.get_k_bvec_neighbors(vecs,'paclitaxel',10))

['paclitaxel', 'bax_gene|bax', 'caspase-3', 'bortezomib', 'etoposide', 'caspase', 'curcumin', 'survivin', 'bcl2-related_protein_11|bcl2l11', 'proto-oncogene_proteins_c-myc|myc']
